# Description:
* In this notebook effectivity of ***1D,2D and 3D CNNs*** is evaluated
* In order to apply CNNs efficiently to the data, it is aranged into arrays in a way that genes with a high covariance are close to each other
* This is done by projecting the data into a 1-,2- or 3 dimensional space using ***t-sne***
* It turns out that this way of aranging the data yields significantly better results for the 1D and 2D CNN
* The 3D CNN generally yields bad results 

# Table of contents
1. [Dimensionality reduction by t-sne](#section-one)
2. [Arranging the data into arrays](#section-two)
3. [Dataloading](#section-three)
4. [Models](#section-four)
5. [Training](#section-five)
6. [Evaluating arrangement and models](#section-six)
    - [Train-, validation and testsets](#subsection-one)
    - [Training models](#subsection-two)
    - [Testing and comparing arangement models](#subsection-three)
7. [Conclusion](#section-seven)

In [ ]:
DoTrain = False
DoTrain3D = False
DoCV = False

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import sklearn
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import OneCycleLR
from sklearn.manifold import TSNE
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
train_features_df = pd.read_csv('../input/lish-moa/train_features.csv')
train_labels_df = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
submission = pd.read_csv('../input/lish-moa/sample_submission.csv')
train_df = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')

In [ ]:
catnames = train_features_df.columns[1:4]
catnames = [str(c) for c in catnames]

contnames = train_features_df.columns[4:]
contnames = [str(c) for c in contnames]

ynames = train_labels_df.columns[1:]
ynames = [y for y in ynames]

joined = pd.concat([train_features_df, train_labels_df], axis=1, join='inner')
joined = joined.drop(columns = 'sig_id')
joined = joined[joined['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)

<a id="section-one"></a>
# 1.) Dimensionality reduction by t-sne

* In the following sections the g- and c-data is aranged into a 1-,2- or 3-dimensional cubic grid of length n_sqrt
* For this the absolute of the entries of the covariance matrix (here named cov) are interpreted as the pairwise distances between the data columns
* When applying t-sne to the absolute covariance matrix the data columns with higher absolute covariance will be closer to each other in the lower diomensional projected space
* The lower diomensional projected space is then divided into equidistantly spaced 'pixels' 
* Then, while iterating over the 'pixels', the data column with the closest distance to the 'pixel' in the projected space is assigned to the respective 'pixel' 
* This assignment to pixels is stored into a dictionary by the GetIndexMapND functions in the Map section
* To these 'images' a CNN is then applied
* As a reference, another 'image' is generated by assigning data columns to each pixel according to the order they appear in the data frame 
* This random arangement of the data columns is supposed determine if the arangement according to covariance is of any use 
* The random arangement is stored into a dictionary by the GetUnityMapND functions in the Map section

In [ ]:
cov = np.cov(train_features_df[train_features_df.columns[4:]].to_numpy().transpose())
plt.imshow(cov)

In [ ]:
cov = 1 / cov

In [ ]:
dim = 2
tsne_em = TSNE(n_components=dim, perplexity=40.0, n_iter=1000, verbose=1, metric='precomputed').fit_transform(np.abs(cov))
if dim == 1:
    plt.hist(*[tsne_em[:,i] for i in range(dim)])
else:
    plt.scatter(*[tsne_em[:,i] for i in range(dim)])

<a id="section-two"></a>
# 2.) Arranging the data into arrays


In [ ]:
def GetIndexMap1D(tsne_em, n_sqrt):
    mini = np.min(tsne_em,axis = 0)
    maxi = np.max(tsne_em,axis = 0)
    d = (maxi - mini) / n_sqrt
    dic={}
    indexhelper = np.arange(0,tsne_em.shape[0])
    tsnecopy = tsne_em.copy()
    count = 0
    for i in range(n_sqrt):        
            pos = mini + np.array([i*d[0]])
            ds = []
            for s in range(tsne_em.shape[0]):
                ds.append(np.linalg.norm(tsnecopy[s] - pos))
            ds = np.array(ds)
            mindindex = indexhelper[ds == np.min(ds)]
            tsnecopy[mindindex] = np.array([np.inf])
            if count <  len(train_df.columns) - 4:
                dic[i] = mindindex[0]
            count += 1
    return dic


def GetIndexMap2D(tsne_em, n_sqrt):
    mini = np.min(tsne_em,axis = 0)
    maxi = np.max(tsne_em,axis = 0)
    d = (maxi - mini) / n_sqrt
    dic={}
    indexhelper = np.arange(0,tsne_em.shape[0])
    tsnecopy = tsne_em.copy()
    count = 0
    for i in range(n_sqrt):
        for j in range(n_sqrt):
            pos = mini + np.array([i*d[0],j*d[1]])
            ds = []
            for s in range(tsne_em.shape[0]):
                ds.append(np.linalg.norm(tsnecopy[s] - pos))
            ds = np.array(ds)
            mindindex = indexhelper[ds == np.min(ds)]
            tsnecopy[mindindex] = np.array([np.inf,np.inf])
            if count <  len(train_df.columns) - 4:
                dic[(i,j)] = mindindex[0]
            count += 1
    return dic


def GetIndexMap3D(tsne_em, n_sqrt):
    mini = np.min(tsne_em,axis = 0)
    maxi = np.max(tsne_em,axis = 0)
    d = (maxi - mini) / n_sqrt
    dic={}
    indexhelper = np.arange(0,tsne_em.shape[0])
    tsnecopy = tsne_em.copy()
    count = 0
    for i in range(n_sqrt):
        for j in range(n_sqrt):
            for k in range(n_sqrt):
                pos = mini + np.array([i*d[0],j*d[1],k*d[2]])
                ds = []
                for s in range(tsne_em.shape[0]):
                    ds.append(np.linalg.norm(tsnecopy[s] - pos))
                ds = np.array(ds)
                mindindex = indexhelper[ds == np.min(ds)]
                tsnecopy[mindindex] = np.array([np.inf,np.inf,np.inf])
                if count <  len(train_df.columns) - 4:
                    dic[(i,j,k)] = mindindex[0]
                count += 1
    return dic

In [ ]:
def GetUnityMap1D(n_sqrt):
    dic = {}
    for i in range(n_sqrt):
        dic[i] = i
    return dic

def GetUnityMap2D(n_sqrt):
    dic = {}
    for i in range(n_sqrt):
        for j in range(n_sqrt):
            dic[(i,j)] = i * n_sqrt + j
    return dic

def GetUnityMap3D(n_sqrt):
    dic = {}
    for i in range(n_sqrt):
        for j in range(n_sqrt):
            for k in range(n_sqrt):
                dic[(i,j,k)] = i * n_sqrt * n_sqrt + j * n_sqrt + k            
    return dic

<a id="section-three"></a>
# 3.) Dataloading

In [ ]:
def getBatchNormalized(inds,dic,mini,maxi,dim):    
    if dim == 1:          
        return getBatchNormalized1D(inds,dic,mini,maxi,872).astype(np.single)
    if dim == 2:
        return getBatchNormalized2D(inds,dic,mini,maxi,30).astype(np.single)
    if dim == 3:
        return getBatchNormalized3D(inds,dic,mini,maxi,10).astype(np.single)
    
def getStats(dic,dim):    
    if dim == 1:          
        return getStats1D(dic,872)
    if dim == 2:
        return getStats2D(dic,30)
    if dim == 3:
        return getStats3D(dic,10)

def getBatchNormalized1D(inds,dic,mini,maxi,n_sqrt):    
        
    image = np.zeros((len(inds),1,n_sqrt))
    rows = joined[contnames].iloc[inds].to_numpy()   
    
    for i in range(n_sqrt):        
        try:
            image[:,0,i] = rows[:,dic[i]]
        except:                
            continue
                
    ampl = (maxi - mini)
    ampl[ampl == 0] = 1
    
    result = (image - mini) / ampl
    
    return result   

def getBatchNormalized2D(inds,dic,mini,maxi,n_sqrt):    
        
    image = np.zeros((len(inds),1,n_sqrt,n_sqrt))
    rows = joined[contnames].iloc[inds].to_numpy()  
    
    for i in range(n_sqrt):
        for j in range(n_sqrt):
            try:
                image[:,0,i,j] = rows[:,dic[(i,j)]]
            except:                
                continue
                
    ampl = (maxi - mini)
    ampl[ampl == 0] = 1
    
    result = (image - mini) / ampl
    
    return result   

def getBatchNormalized3D(inds,dic,mini,maxi,n_sqrt):    
        
    image = np.zeros((len(inds),1,n_sqrt,n_sqrt,n_sqrt))
    rows = joined[contnames].iloc[inds].to_numpy()  
    
    for i in range(n_sqrt):
        for j in range(n_sqrt):
            for j in range(n_sqrt):            
                try:
                    image[:,0,i,j,k] = rows[:,dic[(i,j,k)]]
                except:                
                    continue
                
    ampl = (maxi - mini)
    ampl[ampl == 0] = 1
    
    result = (image - mini) / ampl
    
    return result   

def getStats1D(dic,n_sqrt):
    image = np.zeros((joined.shape[0],1,n_sqrt))
    row = joined[contnames].to_numpy()    
    for i in range(n_sqrt):        
        try:
            image[:,0,i] = row[:,dic[i]]
        except:                
            continue                
                
    return np.min(image,axis = 0), np.max(image,axis = 0)    


def getStats2D(dic,n_sqrt):
    image = np.zeros((joined.shape[0],1,n_sqrt,n_sqrt))
    row = joined[contnames].to_numpy()    
    for i in range(n_sqrt):
        for j in range(n_sqrt):
            try:
                image[:,0,i,j] = row[:,dic[(i,j)]]
            except:                
                continue                
                
    return np.min(image,axis = 0), np.max(image,axis = 0)    

def getStats3D(dic,n_sqrt):
    image = np.zeros((joined.shape[0],1,n_sqrt,n_sqrt,n_sqrt))
    row = joined[contnames].to_numpy()    
    for i in range(n_sqrt):
        for j in range(n_sqrt):
            for k in range(n_sqrt):
                try:
                    image[:,0,i,j,k] = row[:,dic[(i,j,k)]]
                except:                
                    continue                
                
    return np.min(image,axis = 0), np.max(image,axis = 0)    

def getLabelBatch(inds):
    entry = joined.iloc[inds]
    return entry[ynames].to_numpy().astype(np.single)

In [ ]:
class BatchLoader:
    def __init__(self,trainInds,valInds,dim,dic,bs):
        self.trainIndices = trainInds
        self.valInds = valInds
        self.dim = dim
        self.dic = dic
        self.min,self.max = getStats(dic,dim)
        self.its = trainIndices.shape[0] // bs
        self.bs = bs
        if self.trainIndices.shape[0] - self.its * bs > 0:
            self.its += 1
        
    def randomize(self):
        self.pmt = np.random.permutation(range(self.trainIndices.shape[0]))
        
    def __len__(self):
        return self.its      
    
    def getValidationSet(self):        
        inpBatch = getBatchNormalized(self.valInds,self.dic,self.min,self.max,self.dim) 
        tgtBatch = getLabelBatch(self.valInds)    
        
        return torch.from_numpy(inpBatch), torch.from_numpy(tgtBatch)      
        
    def __getitem__(self, index):
        start = index * self.bs
        end = min((index+1)*self.bs,self.trainIndices.shape[0])
        inds = self.trainIndices[self.pmt][start:end]
        inpBatch = getBatchNormalized(inds,self.dic,self.min,self.max,self.dim) 
        tgtBatch = getLabelBatch(inds)    
        
        return torch.from_numpy(inpBatch), torch.from_numpy(tgtBatch)      

<a id="section-four"></a>
# 4.) Models
* Here the 1-,2- and 3-dimensional CNN models are defined
* These models are very simple and consist of two convolutional layers, with number of channels and filter size can be set varied

In [ ]:
class QuickModelBase(nn.Module):
    def __init__(self):
        super(QuickModelBase, self).__init__()  
        
        
    def forward(self, x, dbg = False, dropout = True):        
        x = self.bn1(x)
        x = self.conv1(x)
        x = self.act1(x)
        if dropout: x = self.do1(x)
        
        x = self.bn2(x)
        x = self.conv2(x)
        x = self.act2(x)
        if dropout: x = self.do2(x)
        
        x = self.bn3(x)
        x = self.conv3(x)
        x = self.act3(x)
        if dropout: x = self.do3(x)
        
        x = self.bn4(x)                
        x = torch.flatten(x,start_dim = 1)        
        x = self.full1(x)
        x = self.act4(x)       
                
        return x
    
    
class QuickModel1DGen(QuickModelBase):
    def __init__(self,feat_num,filter_size,n_sqrt):
        super(QuickModel1DGen, self).__init__()
        self.bn1 = torch.nn.BatchNorm1d(1)
        self.conv1 = nn.Conv1d(1,feat_num[0],filter_size[0],1)
        self.act1 = nn.PReLU()
        self.do1 = nn.Dropout(p=0.2)
        self.bn2 = torch.nn.BatchNorm1d(feat_num[0])
        self.conv2 = nn.Conv1d(feat_num[0],feat_num[1],filter_size[1],1)
        self.act2 = nn.PReLU()  
        self.do2 = nn.Dropout(p=0.2)
        self.bn3 = torch.nn.BatchNorm1d(feat_num[1])
        self.conv3 = nn.Conv1d(feat_num[1],feat_num[2],n_sqrt-filter_size[0]-filter_size[1]+2,1)
        self.act3 = nn.PReLU()
        self.do3 = nn.Dropout(p=0.2)
        self.bn4 = torch.nn.BatchNorm1d(feat_num[2])
        self.full1 = torch.nn.Linear(feat_num[2], 206)
        self.act4 = nn.PReLU()
        

class QuickModel2DGen(QuickModelBase):
    def __init__(self,feat_num,filter_size,n_sqrt):
        super(QuickModel2DGen, self).__init__()
        self.bn1 = torch.nn.BatchNorm2d(1)
        self.conv1 = nn.Conv2d(1,feat_num[0],filter_size[0],1)
        self.act1 = nn.PReLU()  
        self.do1 = nn.Dropout(p=0.2)
        self.bn2 = torch.nn.BatchNorm2d(feat_num[0])
        self.conv2 = nn.Conv2d(feat_num[0],feat_num[1],filter_size[1],1)
        self.act2 = nn.PReLU()  
        self.do2 = nn.Dropout(p=0.2)
        self.bn3 = torch.nn.BatchNorm2d(feat_num[1])
        self.conv3 = nn.Conv2d(feat_num[1],feat_num[2],n_sqrt-filter_size[0]-filter_size[1]+2,1)
        self.act3 = nn.PReLU()
        self.do3 = nn.Dropout(p=0.2)
        self.bn4 = torch.nn.BatchNorm2d(feat_num[2])
        self.full1 = torch.nn.Linear(feat_num[2], 206)
        self.act4 = nn.PReLU()    
        
        
class QuickModel3DGen(QuickModelBase):
    def __init__(self,feat_num,filter_size, n_sqrt):
        super(QuickModel3DGen, self).__init__()
        self.bn1 = torch.nn.BatchNorm3d(1)
        self.conv1 = nn.Conv3d(1,feat_num[0],filter_size[0],1)
        self.act1 = nn.PReLU()        
        self.do1 = nn.Dropout(p=0.2)
        self.bn2 = torch.nn.BatchNorm3d(feat_num[0])
        self.conv2 = nn.Conv3d(feat_num[0],feat_num[1],filter_size[1],1)
        self.act2 = nn.PReLU()  
        self.do2 = nn.Dropout(p=0.2)
        self.bn3 = torch.nn.BatchNorm3d(feat_num[1])
        self.conv3 = nn.Conv3d(feat_num[1],feat_num[2],n_sqrt-filter_size[0]-filter_size[1]+2,1)
        self.act3 = nn.PReLU() 
        self.do3 = nn.Dropout(p=0.2)
        self.bn4 = torch.nn.BatchNorm3d(feat_num[2])
        self.full1 = torch.nn.Linear(feat_num[2], 206)
        self.act4 = nn.PReLU()

<a id="section-five"></a>
# 5.) Training

In [ ]:
def TrainEpoch(valloss,model,opt,sch,dim,dl,valInds,loss,istest,fold=None):
    count = 0
    modelname = f"BestModel{dim}D" if not istest else f"BestRefModel{dim}D"
    if fold != None: modelname = modelname + f"Fold{fold}"
    modelname = modelname + ".pth"
    
    for i in range(len(dl)): 
        (inp,tgt) = dl[i]
        opt.zero_grad()       
        fwd = model(inp.cuda())
        ys = tgt.cuda()
        output = loss(fwd, ys)        
        output.backward()
        opt.step()   
        
        if count % 1 == 0:           
            batchVal,ysVal = dl.getValidationSet()
            fwdVal = model(batchVal.cuda())            
            currentvalloss = loss(fwdVal, ysVal.cuda()).detach().cpu().numpy()   
            
            if currentvalloss < valloss:
                valloss = currentvalloss
                torch.save(model.state_dict(), modelname)
                
        count += 1
                
    return currentvalloss

In [ ]:
def Train(dim, dic, bs, trainInds, valInds, model,istest,fold = None):    
    dl = BatchLoader(trainInds,valInds, dim, dic,bs)

    loss = nn.BCEWithLogitsLoss().cuda()    

    lrstart = 0.1
    optimizer = optim.Adam(model.parameters(), lr=lrstart)
    scheduler = ReduceLROnPlateau(optimizer, 'min')

    valloss = np.inf
    for e in range(70):
        if e % 3 == 0: print('Epoch: ' + str(e)) 
        dl.randomize()
        currentvalloss = TrainEpoch(valloss,model,optimizer,scheduler,dim,dl,valIndices,loss,istest,fold)
        scheduler.step(currentvalloss)
        if abs(currentvalloss - valloss) < 0.0000001: break
        if currentvalloss < valloss: 
            valloss = currentvalloss
            print('BestLoss: ' + str(currentvalloss))
    
        

<a id="section-six"></a>
# 6.) Evaluating arrangement and models

<a id="subsection-one"></a>
# 6.1) Train-, validation and testsets 

In [ ]:
folds = joined.copy()
Fold = MultilabelStratifiedKFold(n_splits=5,shuffle=True,random_state=42)

indices = {}
for n, (trainidx, validx) in enumerate(Fold.split(joined, joined[ynames])):    
    indices[n] = (trainidx, validx)

In [ ]:
valIndices = indices[0][1]
testIndices = indices[4][1]
trainIndices =  indices[0][0]
trainIndices = trainIndices[np.isin(trainIndices,testIndices,invert=True)]
valIndices = valIndices[np.isin(valIndices,testIndices,invert=True)]

<a id="subsection-two"></a>
# 6.2) Training models

# 1D

In [ ]:
if DoTrain or DoCV:
    tsne1D = TSNE(n_components=1, perplexity=40.0, n_iter=1000, verbose=1, metric='precomputed').fit_transform(np.abs(cov))
    np.save('tsne1D',tsne1D)
else: 
    tsne1D = np.load('../input/t-sne-for-applying-nd-cnns/tsne1D.npy')    

In [ ]:
dicRef1D = GetUnityMap1D(872)
dic1D = GetIndexMap1D(tsne1D,872)

In [ ]:
bs = 512
modelRef1D = QuickModel1DGen([10,20,40],[5,30],872).cuda()
if DoTrain: Train(1, dicRef1D, bs, trainIndices, valIndices, modelRef1D,True)

In [ ]:
model1D = QuickModel1DGen([10,20,40],[5,30],872).cuda()
if DoTrain: Train(1, dic1D, bs, trainIndices, valIndices, model1D,False)

In [ ]:
if DoCV:
    fold = 0
    for i in indices:
        print(f"Fold: {i}")
        model1D = QuickModel1DGen([10,20,40],[5,30],872).cuda()
        valInds = indices[i][1]
        trainInds = indices[i][0]
        Train(1, dicRef1D, bs, trainInds, valInds, model1D,True,fold)
        fold += 1

# 2D

In [ ]:
if DoTrain or DoCV:
    tsne2D = TSNE(n_components=2, perplexity=40.0, n_iter=1000, verbose=1, metric='precomputed').fit_transform(np.abs(cov))
    np.save('tsne2D',tsne2D)
else: tsne2D = np.load('../input/t-sne-for-applying-nd-cnns/tsne2D.npy')    

In [ ]:
dicRef2D = GetUnityMap2D(30)
dic2D = GetIndexMap2D(tsne2D,30)

In [ ]:
bs = 512
modelRef2D = QuickModel2DGen([10,20,40],[10,10],30).cuda()
if DoTrain: Train(2, dicRef2D, bs, trainIndices, valIndices, modelRef2D,True)

In [ ]:
model2D = QuickModel2DGen([10,20,40],[10,10],30).cuda()
if DoTrain: Train(2, dic2D, bs, trainIndices, valIndices, model2D,False)

In [ ]:
if DoCV:
    fold = 0
    for i in indices:
        print(f"Fold: {i}")
        model2D = QuickModel2DGen([10,20,40],[10,10],30).cuda()
        valInds = indices[i][1]
        trainInds = indices[i][0]
        Train(2, dic2D, 512, trainInds, valInds, model2D,False,fold)
        fold += 1

# 3D

In [ ]:
if DoTrain3D:
    tsne3D = TSNE(n_components=3, perplexity=40.0, n_iter=1000, verbose=1, metric='precomputed').fit_transform(np.abs(cov))
    np.save('tsne3D',tsne3D)
else: tsne3D = np.load('../input/t-sne-for-applying-nd-cnns/tsne3D.npy')    

In [ ]:
dicTest3D = GetUnityMap3D(10)
dic3D = GetIndexMap3D(tsne3D,10)

In [ ]:
bs = 64
modelRef3D = QuickModel3DGen([20,40,80],[3,5],10).cuda()
if DoTrain3D: Train(3, dicTest3D, bs, trainIndices, valIndices, modelRef3D,True)

In [ ]:
model3D = QuickModel3DGen([10,20,40],[3,5],10).cuda()
if DoTrain3D: Train(3, dic3D, 256, trainIndices, valIndices, model3D,False)

<a id="subsection-three"></a>
# 6.3) Testing and comparing arangement models

# 1D

In [ ]:
mini, maxi = getStats(dic1D,1)
batch1D = getBatchNormalized(testIndices,dic1D,mini,maxi,1) 
mini, maxi = getStats(dicRef1D,1)
batchRef1D = getBatchNormalized(testIndices,dicRef1D,mini,maxi,1) 

In [ ]:
if DoTrain: 
    model1D.load_state_dict(torch.load("./BestModel1D.pth"))
    modelRef1D.load_state_dict(torch.load("./BestRefModel1D.pth"))
else: 
    model1D.load_state_dict(torch.load("../input/t-sne-for-applying-nd-cnns/BestModel1D.pth"))
    modelRef1D.load_state_dict(torch.load("../input/t-sne-for-applying-nd-cnns/BestRefModel1D.pth"))
    
loss = nn.BCEWithLogitsLoss().cuda()    

In [ ]:
batch1DTorch = torch.from_numpy(batch1D.astype(np.single)).cuda()
batchRef1DTorch = torch.from_numpy(batchRef1D.astype(np.single)).cuda()
fwdVal = model1D.forward(batch1DTorch,False,False)
fwdRefVal = modelRef1D.forward(batchRef1DTorch,False,False)
ysVal = torch.from_numpy(getLabelBatch(testIndices)).cuda()
testloss1D = loss(fwdVal, ysVal).detach().cpu().numpy()
testlossRef1D = loss(fwdRefVal, ysVal).detach().cpu().numpy()

In [ ]:
print('1D: ')
print('Test loss: ')
print(testloss1D)
print('Test loss unaranged reference model: ')
print(testlossRef1D)

# 2D

In [ ]:
mini, maxi = getStats(dic2D,2)
batch2D = getBatchNormalized(testIndices,dic2D,mini,maxi,2) 
mini, maxi = getStats(dicRef2D,2)
batchRef2D = getBatchNormalized(testIndices,dicRef2D,mini,maxi,2) 

In [ ]:
if DoTrain:
    model2D.load_state_dict(torch.load("./BestModel2D.pth"))
    modelRef2D.load_state_dict(torch.load("./BestRefModel2D.pth"))
else:
    model2D.load_state_dict(torch.load("../input/t-sne-for-applying-nd-cnns/BestModel2D.pth"))
    modelRef2D.load_state_dict(torch.load("../input/t-sne-for-applying-nd-cnns/BestRefModel2D.pth"))
    
loss = nn.BCEWithLogitsLoss().cuda()    

In [ ]:
batch2DTorch = torch.from_numpy(batch2D.astype(np.single)).cuda()
batchRef2DTorch = torch.from_numpy(batchRef2D.astype(np.single)).cuda()
fwdVal = model2D.forward(batch2DTorch,False,False)
fwdRefVal = modelRef2D.forward(batchRef2DTorch,False,False)
ysVal = torch.from_numpy(getLabelBatch(testIndices)).cuda()
testloss2D = loss(fwdVal, ysVal).detach().cpu().numpy()
testlossRef2D = loss(fwdRefVal, ysVal).detach().cpu().numpy()

In [ ]:
print('2D: ')
print('Test loss: ')
print(testloss2D)
print('Test loss unaranged reference model: ')
print(testlossRef2D)

<a id="section-seven"></a>
# 7.) Conclusion & Outlook

* The arangement of the data columns into 2D arrays has a large impact on the performance of 1D and 2D CNNs
* The method of aranging the data into multidimensional arrays investigated here seems promising
* Overall the 1D CNN tried out here seems to perform slightly better then the 2D CNN
* It should be investigated if this holds for deeper models and other train-/testset combinations
* 3D CNN perform poorly